In [1]:
%pip install -U --quiet langchain-core langchain-community==0.2.6 youtube_search Wikipedia grandalf langchain-ollama



Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template('''Tell me the name of latest {genre} movie in which {actor} is one of the actors. 
  Output the movie name in the form of a json e.g. {{"movie_name": "movie name"}}. 
  Output only the json without any additional information or side notes.
 ''')
 
 # test
prompt_template.format(genre="drama", actor="Brad Pitt")

'Tell me the name of latest drama movie in which Brad Pitt is one of the actors. \n  Output the movie name in the form of a json e.g. {"movie_name": "movie name"}. \n  Output only the json without any additional information or side notes.\n '

In [3]:
from langchain_ollama import ChatOllama 
import json

def invoke_llm(prompt):
    llm = ChatOllama(
        model="llama3",
        temperature=0.6, 
        max_tokens=512
    )
    ai_message = llm.invoke(prompt)
    return ai_message.content

def extract_movie (text:str) -> str:
    return json.loads(text)['movie_name']

# test LLM
invoke_llm(prompt_template.format(genre="Action", actor="Al Pacino"))


'{"movie_name": "Misbehavior"}'

In [4]:
movie = extract_movie(invoke_llm(prompt_template.format(genre="Action", actor="Al Pacino")))
movie

'The Irishman'

In [5]:
from langchain_community.retrievers import WikipediaRetriever

def retrieve_movie_info(movie):
  retriever = WikipediaRetriever()
  return retriever.invoke(input=f"hollywood english movie: {movie}")

# test
docs = retrieve_movie_info("House of Gucci")
print(docs[0].page_content[:100])

House of Gucci is a 2021 American biographical crime drama film directed by Ridley Scott, based on t


In [6]:
from langchain_community.tools import YouTubeSearchTool

def get_youtube_trailer(movie):
  tool = YouTubeSearchTool()
  return tool.run(f"{movie} movie trailer").split("'")[1]


#test 
get_youtube_trailer("The Shawshank Redemption")

'https://www.youtube.com/watch?v=PLl99DlL6b4&pp=ygUmVGhlIFNoYXdzaGFuayBSZWRlbXB0aW9uIG1vdmllIHRyYWlsZXI%3D'

In [7]:
from langchain_core.output_parsers import StrOutputParser

recommend_movie_chain =  prompt_template | invoke_llm | StrOutputParser() | extract_movie 

# test chain 1
recommend_movie_chain.invoke(input={"genre":"action", "actor":"al pacino"})

'The Irishman'

In [13]:
wiki_template = PromptTemplate.from_template(
    """ 
    Summarize the below movie description in a few sentences, only output the summary nothing else:: 
    
    {wiki_dump}
    
    """
)

In [14]:
final_response = PromptTemplate.from_template(
  """ 
    Combine the results from movie name, movie plot description and youtube trailer link below into  a well written movie recommendation::
    
    {movie_plot}
    {youtube_trailer}
    """
)

In [15]:
movie_plot_chain = retrieve_movie_info | wiki_template | invoke_llm | StrOutputParser()

In [16]:
youtube_trailer_chain = get_youtube_trailer

In [17]:
chain_test = (recommend_movie_chain | StrOutputParser() )
          
print(chain_test.invoke(input={"genre":"action", "actor":"al pacino"}))

The Irishman


In [18]:
chain = (
          recommend_movie_chain | StrOutputParser()
          | {"movie_plot": movie_plot_chain , "youtube_trailer": youtube_trailer_chain} 
          | final_response 
          | invoke_llm       
        )


In [21]:
print(chain.invoke(input={"genre":"action", "actor":"Tom Cruise"}))

Here's a well-written movie recommendation combining the results:

**Movie Recommendation:** Top Gun (1986)

Get ready to experience the thrill of high-octane action and heart-pumping drama with Tom Cruise in this iconic 80s classic, Top Gun! This 1986 film tells the story of Lieutenant Pete "Maverick" Mitchell, a young naval aviator who enrolls at the prestigious US Navy's Fighter Weapons School (Top Gun) to hone his skills alongside his trusty radar intercept officer, Goose. With its stunning visual effects and memorable soundtrack, Top Gun has become a cultural phenomenon that continues to captivate audiences to this day.

Watch the official trailer below to get a taste of the excitement:

https://www.youtube.com/watch?v=qSqVVswa420&pp=ygUfVG9wIEd1bjogTWF2ZXJpY2sgbW92aWUgdHJhaWxlcg%3D%3D

While initially receiving mixed reviews, Top Gun went on to become a huge commercial success and earned critical acclaim for its groundbreaking visual effects and memorable soundtrack. Don't miss t

In [22]:
chain.get_graph().print_ascii()

                      +-------------+                      
                      | PromptInput |                      
                      +-------------+                      
                             *                             
                             *                             
                             *                             
                    +----------------+                     
                    | PromptTemplate |                     
                    +----------------+                     
                             *                             
                             *                             
                             *                             
                      +------------+                       
                      | invoke_llm |                       
                      +------------+                       
                             *                             
                             *          